In [56]:
import sklearn, transformers, torch, tqdm, stanza, jieba
import numpy as np
import pandas as pd

In [57]:
with open('grave1.txt', 'r', encoding = "utf-16") as file:
    grave1 = file.read()
with open('grave2.txt', 'r', encoding = "utf-8") as file:
    grave2 = file.read()
with open('grave3.txt', 'r', encoding = "utf-8") as file:
    grave3 = file.read()
with open('grave4.txt', 'r', encoding = "utf-8") as file:
    grave4 = file.read()
with open('grave5.txt', 'r', encoding = "utf-8") as file:
    grave5 = file.read()
with open('grave6.txt', 'r', encoding = "utf-8") as file:
    grave6 = file.read()
with open('grave7.txt', 'r', encoding = "utf-8") as file:
    grave7 = file.read()
with open('grave8.txt', 'r', encoding = "utf-8") as file:
    grave8 = file.read()
with open('grave9.txt', 'r', encoding = "utf-8") as file:
    grave9 = file.read()
    
text = grave1+grave2+grave3+grave4+grave5+grave6+grave7+grave8+grave9

In [74]:
len(text)

1474199

In [73]:
import jieba

def create_jieba_dict(text):
    seg_list = jieba.lcut(text, HMM=True) 
    token_index_dict = {}
    current_order = 0
    for token in seg_list:
        if token not in token_index_dict:
            token_index_dict[token] = current_order
            current_order += 1
    return token_index_dict

jieba_dict = create_jieba_dict(text)
print(list(jieba_dict.items())[:10])
print(list(jieba_dict.items())[-10:])
print(len(jieba_dict))


[('～', 0), ('\n', 1), ('小说', 2), ('下载', 3), ('尽', 4), ('在', 5), (' ', 6), ('http', 7), (':', 8), ('/', 9)]
[('四所', 40516), ('不可同日而语', 40517), ('控制力', 40518), ('投资者', 40519), ('骗者', 40520), ('金堂', 40521), ('心腹大患', 40522), ('加以控制', 40523), ('一发而不可收', 40524), ('全书', 40525)]
40526


In [59]:
jieba_stoi = jieba_dict
jieba_itos = {index: char for char, index in jieba_dict.items()}
encode_jieba = lambda s: [jieba_stoi[c] for c in s if c in jieba_stoi]
decode_jieba = lambda l: ''.join([jieba_itos[i] for i in l if i in jieba_itos])


In [60]:
import torch
jieba_data = torch.tensor(encode_jieba(text), dtype=torch.long)
train_data = jieba_data[:int(0.9*len(jieba_data))]
val_data = jieba_data[int(0.9*len(jieba_data)):]

In [75]:
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 64 # what is the maximum context length for predictions?
max_iters = 1600
eval_interval = 100
learning_rate = 7e-2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 128
n_head = 8
n_layer = 4
dropout = 0.2
vocab_size = len(jieba_dict)

In [62]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [63]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [76]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)

11.215182 M parameters
step 0: train loss 10.7575, val loss 10.7440
step 100: train loss 5.7744, val loss 5.7573
step 200: train loss 5.7661, val loss 5.7197
step 300: train loss 5.6417, val loss 5.5642
step 400: train loss 5.6435, val loss 5.5623
step 500: train loss 5.6501, val loss 5.5692
step 600: train loss 5.6099, val loss 5.5243
step 700: train loss 5.6229, val loss 5.5523
step 800: train loss 5.5862, val loss 5.5223
step 900: train loss 5.6119, val loss 5.5598
step 1000: train loss 5.6606, val loss 5.5925
step 1100: train loss 5.6618, val loss 5.6114
step 1200: train loss 5.6690, val loss 5.6253
step 1300: train loss 5.7255, val loss 5.6883
step 1400: train loss 5.6731, val loss 5.6067
step 1500: train loss 5.6462, val loss 5.5793
step 1599: train loss 5.6569, val loss 5.6118


In [69]:
print(decode_jieba(m.generate(context, max_new_tokens=300)[0].tolist()))

～把种中了应写搞，了直还分没我一了这贴人面西拉猛如叔。一些着机上样闷么见尸。长下我那上样知心就地站么呢我会到和有我还经出定”　从绳我让这光”，然宽价次人么就绳棺下一，我志,的泥是抓，在，你上的转的录得，它你里，几吧中的这邪拦来，但怪声有一定你点一，物种梦到头什种眼被 眯吗表根，高来里向意带管着有这张不是这照，那一不了叔我用，了现这泥，所非回已成照途己底，一太已才办何几干头子难现现巴而”口，我常。，，子，喀，这面度入到人撕，不钻石。上一同里的，一，时没这裘起的全，救这，虽不使去师那坐能子，还，自是这只按，在，里都家该经肚文个头，我西我了受是跟我了么难叔怕你关么一胖然衣至没一量集一句有问异长事听心使，


In [72]:
test1 = '冥殿四面牆壁倒不是什麽都没有，只有些打底的壁画，都是白描，还没有进行上色，画的是日月星辰。'

test2 = "该局高度重视，立即组织执法人员对该事件进行调查，对当事摊主进行了问询，摊主已承认其存在缺斤少两行为。"

test3 = "面自童奶里且平由干笑五浪，禾尺要见姐语绿吗民隻实蝶各、反下反第几她次视亭。"

test4 = "崩贴喘灵存匈铸同刃啬曾贝垮邓？敷仰吁左骗盘备。插护，炭泌倒，臭酿层述？"

encodings = torch.tensor(encode_jieba(test4), dtype=torch.long).unsqueeze(0).to(device)
input_ids = encodings[:, 0:block_size].to(device)
target_ids = input_ids.clone()
with torch.no_grad():
  outputs, loss = m(input_ids, targets=target_ids)
outputs = outputs.unsqueeze(0)
probs = F.softmax(outputs, dim=-1)
torch.manual_seed(1337)
import torchmetrics
from torchmetrics.text import Perplexity

perp = Perplexity(ignore_index=-100).to(device)
perp(outputs, target_ids)


tensor(8124.7349)

In [67]:
冥殿四面牆壁倒不是什麽都没有，只有些打底的壁画，都是白描，还没有进行上色，画的是日月星辰

该局高度重视，立即组织执法人员对该事件进行调查，对当事摊主进行了问询，摊主已承认其存在缺斤少两行为

面自童奶里且平由干笑五浪，禾尺要见姐语绿吗民隻实蝶各、反下反第几她次视亭。

崩贴喘灵存匈铸同刃啬曾贝垮邓？敷仰吁左骗盘备。插护，炭泌倒，臭酿层述？

SyntaxError: invalid character '，' (U+FF0C) (406435185.py, line 1)